# `Nuqleon.Memory`

Provides object pools, function memoization, and caching utilities.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [1]:
#r "bin/Debug/net50/Nuqleon.Memory.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [1]:
#r "nuget:Nuqleon.Memory,*-*"

## Using object pools

Object pools can be used to reduce the overhead of allocating fresh objects. This library supports object pooling for arbitrary types but also have built-in support for commonly used types, such as `StringBuilder` and various collection types.

First, we'll explore support for built-in types by having a look at pooling for `Stack<T>` objects. Support for other types is completely analogous.

### Step 1 - Create a pool

The first step is to create a pool using one of the `Create` static method overloads on the pool type. In this example, we'll use `StackPool<T>` and create a pool that can grow up to `8` instances.

In [1]:
var pool = StackPool<int>.Create(size: 8);

### Step 2 - Inspect the pool using `DebugView`

At any time we can have a look at the pool's internals using the `DebugView` property. This shows statistics of the pool, as well as call stacks for allocations and deallocations in `DEBUG` builds.

In [1]:
pool.DebugView

### Step 3 - Allocate an object from the pool

One way to allocate an object from the pool is by using `Allocate`. Once we're done using the object, we call `Free`. This is typically done in a safe manner, e.g. using a `try...finally...` statement. In case an object is not returned to the pool, it will just get garbage collected and the pool's performance will be degraded. However, there won't be a memory leak.

In [1]:
PooledStack<int> stack = pool.Allocate();

Console.WriteLine(pool.DebugView);

try
{
    // Use the object here.
    stack.Push(1);
}
finally
{
    pool.Free(stack);
}

Now that we've returned the object back to the pool, let's inspect the pool again.

In [1]:
Console.WriteLine(pool.DebugView);

### Step 4 - An alternative way to allocate from the pool

An alternative way to allocate an object from the pool is by using `New` which returns a holder object that implements `IDisposable` and can be used with a `using` statement. This makes it easier to ensure returning the object to the pool, even in exceptional circumstances.

In [1]:
using (PooledStackHolder<int> h = pool.New())
{
    display(pool.DebugView);
    
    var s = h.Stack;

    // Use the object here.
    s.Push(1);
}

Now that we've returned the object back to the pool, let's inspect the pool again.

In [1]:
display(pool.DebugView)

## Function memoization

Function memoization is a technique to cache the results of evaluating a pure function in order to speed up future invocations. As an example, consider the well-known recursive Fibonacci generator.

In [1]:
Func<long, long> fib = null;

fib = n => n <= 1 ? 1 : checked(fib(n - 1) + fib(n - 2));

Evaluating the Fibonacci generator causes repeated evaluation of the same function with the same argument. For example:

```
fib(3) = fib(2) + fib(1)
fib(2) =          fib(1) + fib(0)
```

Let's run the Fibonacci generator for a few values and time the execution.

In [1]:
using System.Diagnostics;

void PrintFibonacci(int max, TimeSpan maxTimeToCompute)
{
    var sw = new Stopwatch();

    for (int i = 0; i < max; i++)
    {
        sw.Restart();

        long res = 0L;
        try
        {
            res = fib(i);
        }
        catch (OverflowException)
        {
            Console.WriteLine($"fib({i}) = Overflow");
            return;
        }

        sw.Stop();
        Console.WriteLine($"fib({i}) = {res} - Took {sw.Elapsed}");

        // Stop if it starts taking too long.
        if (sw.Elapsed > maxTimeToCompute)
        {
            break;
        }
    }
}

PrintFibonacci(100, TimeSpan.FromSeconds(5));

Most likely, you didn't get much further than some `40`-ish iterations. Let's use this example to illustrate memoization for function evaluation.

The first step to make memoization work is to create a so-called *memoization cache factory*. Each memoized function will have an associated memoization cache. Factories for such caches determine the policy of the cache. In the sample below we'll use an `Unbounded` cache which does not limit the number of entries in the cache. Other options are caches with least-recently-used (LRU) policies or other eviction policies.

In [1]:
using System.Memory;

IMemoizationCacheFactory factory = MemoizationCacheFactory.Unbounded;

Now that we have a cache factory, we can create a *memoizer* that will be used to memoize functions.

In [1]:
IMemoizer mem = Memoizer.Create(factory);

Finally, we use the memoizer to `Memoize` the function. After doing so, we end up with a pair of a cache and a memoized function of the same delegate type.

In [1]:
IMemoizedDelegate<Func<long, long>> memoizedFib = mem.Memoize(fib);

// The cache and delegate pair.
IMemoizationCache cache = memoizedFib.Cache;
Func<long, long> fibMemoized = memoizedFib.Delegate;

// Let's replace the original delegate by the memoized one, which was also used in the body of the recursive definition of fib.
fib = fibMemoized;

// Now we should get much further along.
PrintFibonacci(100, TimeSpan.FromSeconds(5));

To see what's going on, let's explore the cache.

In [1]:
cache.DebugView

Other memoization functionality includes:

* Support for memoization of N-ary functions.
* Different cache policies, e.g. LRU or eviction based on metrics. See static methods on `MemoizationCacheFactory`.
* Weak memoization caches where inputs of the function are not held alive for the sole purpose of caching (i.e. they act as weak references).